In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
from re import sub
# get data
def get_data(df, column, train_end=-1, days_before=300, days_pred=7):

    new_df = pd.DataFrame(pd.concat([df[column[0]], df[column[1]]], axis = 1))

    # Generate training set
    data = pd.DataFrame()
    
    # prepare train set
    values_list = new_df.values.tolist()
    for i in range(days_before):
        # the (-days_before - days_pred) days are used to do prediction
        data['b%d' % i] = values_list[i: -days_before - days_pred + i]
        

    # prediction set
    for i in range(days_pred):
        data['y%d' % i] = values_list[days_before + i: - days_pred + i]
    
    
    train_data, val_data, test_data = data[:train_end], data[train_end-300:train_end], data[train_end:]
    return train_data, val_data, test_data, new_df.values, df.index.tolist()


# LSTM model
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=2,
            hidden_size=128,
            num_layers=2, 
            batch_first=True)
        
        self.out = nn.Sequential(
            nn.Linear(128, 1))
        
    def forward(self, x):
        r_out, (h_n, h_c) = self.lstm(x, None)
        out = self.out(r_out[:, -7:, :])
        
        return out

class TrainSet(Dataset):
    def __init__(self, data):
        self.data, self.label = data[:, :-7].float(), data[:, -7:].float()

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

def train(exchange_dataset, interest_dataset):
    # Paths
    FILE_NAME = exchange_dataset
    FILE_NAME2 = interest_dataset
    MODEL_PATH = './gdrive/MyDrive/LSTM_foreign_exchange/train_models/' + FILE_NAME + '_lstm.pkl'
    FIGURE_PATH = './gdrive/MyDrive/LSTM_foreign_exchange/train_figures/'

    # Hyperparameters
    LR = 0.0001
    EPOCH = 2500
    TRAIN_END=-1
    DAYS_BEFORE=300
    DAYS_PRED=7

    # read data from data file
    df = pd.read_csv('./gdrive/MyDrive/LSTM_foreign_exchange/data/'+FILE_NAME+'.csv', index_col=0)

    df2 = pd.read_csv('./gdrive/MyDrive/LSTM_foreign_exchange/data/'+FILE_NAME2+'.csv', index_col=0)

    df = df.merge(df2, how='inner', on='Series ID')

    df.index = list(map(lambda x:datetime.datetime.strptime(x, '%d-%b-%y'), df.index))
    df2.index = list(map(lambda x:datetime.datetime.strptime(x, '%d-%b-%y'), df2.index))



    # generate data set use getData method
    train_data, val_data, test_data, all_series, df_index = get_data(df, ['FXRUSD','FIRMMCTRI'], days_before=DAYS_BEFORE, days_pred=DAYS_PRED, train_end=TRAIN_END) # use close price as input
  
    # get all original data
    original_data = np.array(all_series.copy().tolist())
    
    # plot for the original data
    plt.figure(figsize=(12,8))
    plt.plot(df_index, original_data[:,0], label='original_exchange')
    plt.savefig(FIGURE_PATH + FILE_NAME + '_original.png')
    plt.close()
    
    plt.figure(figsize=(12,8))
    plt.plot(df_index, original_data[:,1], label='original_interest')
    plt.savefig(FIGURE_PATH + FILE_NAME2 + '_original.png')
    plt.close()
    
    
    
    # normalization
    # train_data_numpy = np.array(train_data)
    train_data_numpy_d1 = np.array(train_data.values.tolist())[:,:,0]
    train_mean_d1 = np.mean(train_data_numpy_d1)
    train_std_d1  = np.std(train_data_numpy_d1)
    train_data_numpy_d1 = (train_data_numpy_d1 - train_mean_d1) / train_std_d1

    train_data_numpy_d2 = np.array(train_data.values.tolist())[:,:,1]
    train_mean_d2 = np.mean(train_data_numpy_d2)
    train_std_d2  = np.std(train_data_numpy_d2)
    train_data_numpy_d2 = (train_data_numpy_d2 - train_mean_d2) / train_std_d2
    
    train_data_numpy = []
    for i in range(len(train_data_numpy_d1)):
      z = tuple(zip(train_data_numpy_d1[i],train_data_numpy_d2[i]))
      temp = []
      for j in z:
        temp.append(tuple(j))
      
      train_data_numpy.append(temp)

    train_data_numpy = np.array(train_data_numpy)

    train_data_tensor = torch.Tensor(train_data_numpy)
    
    val_data_numpy_d1 = np.array(val_data.values.tolist())[:,:,0]
    val_data_numpy_d1 = (val_data_numpy_d1 - train_mean_d1) / train_std_d1

    val_data_numpy_d2 = np.array(val_data.values.tolist())[:,:,1]
    val_data_numpy_d2 = (val_data_numpy_d2 - train_mean_d2) / train_std_d2

    val_data_numpy = []
    for i in range(len(val_data_numpy_d1)):
      z = tuple(zip(val_data_numpy_d1[i],val_data_numpy_d2[i]))
      temp = []
      for j in z:
        temp.append(tuple(j))
      
      val_data_numpy.append(temp)

    val_data_numpy = np.array(val_data_numpy)
   

    val_data_tensor = torch.Tensor(val_data_numpy)

    # create dataloader
    train_set = TrainSet(train_data_tensor)
    train_loader = DataLoader(train_set, batch_size=256, shuffle=True)

    val_set = TrainSet(val_data_tensor)
    val_loader = DataLoader(val_set, batch_size=256, shuffle=True)

    lstm = LSTM()

    if torch.cuda.is_available():
        lstm = lstm.cuda()

    optimizer = torch.optim.Adam(lstm.parameters(), lr=LR)

    loss_func = nn.MSELoss()

    best_loss = 1000 

    for step in range(EPOCH):
        for tx, ty in train_loader:
            if torch.cuda.is_available():
                tx = tx.cuda()
                ty = ty.cuda() 
            print(torch.squeeze(tx).shape)
            output = lstm(torch.unsqueeze(tx, dim=2))             
            loss = loss_func(torch.squeeze(output), ty)        
            optimizer.zero_grad()  # clear gradients for this training step
            loss.backward()  # back propagation, compute gradients
            optimizer.step()
            
            print('%s epoch : %d  ' % (FILE_NAME, step), 'train_loss : %.4f' % loss.cpu().item())
            
        with torch.no_grad():
            for tx, ty in val_loader:
                if torch.cuda.is_available():
                    tx = tx.cuda()
                    ty = ty.cuda() 
                output = lstm(torch.unsqueeze(tx, dim=2))             
                loss = loss_func(torch.squeeze(output), ty)
            
                print('%s epoch : %d  ' % (FILE_NAME, step), 'val_loss : %.4f' % loss.cpu().item())
            
            if loss.cpu().item() < best_loss:
                best_loss = loss.cpu().item()
                torch.save(lstm, MODEL_PATH.format(loss.cpu().item()))
                print('{} new model saved at epoch {} with val_loss {}'.format(FILE_NAME, step, best_loss))

    torch.save(lstm, MODEL_PATH.format(loss.cpu().item()))

    data_train = []

    #Test and plot prediction figure
    # start index of test
    test_start = len(original_data) + TRAIN_END

    # normalization
    original_data = (original_data - train_mean_d1) / train_std_d1
    original_data = torch.Tensor(original_data)

    for i in range(DAYS_BEFORE, len(original_data) - DAYS_PRED, DAYS_PRED):
        x = original_data[i - DAYS_BEFORE:i]
        x = torch.unsqueeze(torch.unsqueeze(x, dim=0), dim=2)
        
        if torch.cuda.is_available():
            x = x.cuda()

        y = torch.squeeze(lstm(x))
        
        if i < test_start:
            data_train.append(torch.squeeze(y.cpu()).detach().numpy() * train_std_d1 + train_mean_d1)
            
    data_train = np.concatenate(data_train, axis=0)

    # plot result
    plt.figure(figsize=(12,8))
    plt.plot(df_index, original_data.clone().numpy() * train_std_d1 + train_mean_d1, 'r', label='Original')
    plt.plot(df_index[DAYS_BEFORE: len(data_train) + DAYS_BEFORE], data_train, 'b', label='Train')
    plt.legend()
    plt.savefig(FIGURE_PATH + FILE_NAME + '_result.pdf')
    plt.close()
 

train('exchange_rate','interest_rate')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


torch.Size([256, 300, 2])


RuntimeError: ignored